# 0. Installing PySpark in Google Colab

Install Dependencies (needs to be done once each time you re-open this notebook):

1.   Java 8
2.   Apache Spark with hadoop and
3.   Findspark (used to locate the spark in the system)

In [1]:
# install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# install spark (change the version number if needed)
!wget -q https://dlcdn.apache.org/spark/spark-3.5.3/spark-3.5.3-bin-hadoop3.tgz

# unzip the spark file to the current folder
!tar xf spark-3.5.3-bin-hadoop3.tgz

# set your spark folder to your system path environment.
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.3-bin-hadoop3"

# install findspark using pip
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# 1. Wordcount

In [2]:
import sys, os
if 'google.colab' in sys.modules:
    # find automatically the path of the folder containing "file_name" :
    file_name = 'Lab0-part1.ipynb'
    import subprocess
    path_to_file = subprocess.check_output('find . -type f -name ' + str(file_name), shell=True).decode("utf-8")
    path_to_file = path_to_file.replace(file_name,"").replace('\n',"")
    # if previous search failed or too long, comment the previous line and simply write down manually the path below :
    #path_to_file = '/content/gdrive/My Drive/CS5344_AY2425Sem2_Lab'
    print(path_to_file)

./gdrive/MyDrive/CS5344_Lab/CS5344_AY2425Sem2_Lab/


In [3]:
import re

sc = spark.sparkContext

lines = sc.textFile(f'{path_to_file}/Lab0_part1_in.txt')
words = lines.flatMap(lambda l: re.split(r'[^\w]+',l))
pairs = words.map(lambda w: (w, 1))
counts = pairs.reduceByKey(lambda n1, n2: n1 + n2)

print(counts.take(5))

counts.repartition(1).saveAsTextFile(f'{path_to_file}/Lab0_part1_out')
sc.stop()


[('are', 2), ('as', 8), ('look', 1), ('', 4), ('walk', 1)]
